In [3]:
import mne
import moabb
import numpy as np
from moabb.datasets import BNCI2014_001
dataset = BNCI2014_001()
sessions = dataset.get_data(subjects=[1])
subject = 1
session_name = "0train"
run_name = "0"
raw = sessions[subject][session_name][run_name]
raw=raw.pick_types(eeg=True,exclude='bads')

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [4]:


def segment_labels(m, l, s, raw):
    n = (m - l) // s + 1
    labels = []
    
    for i in range(n):
        start = i * s
        end = start + l
        # Convertir les indices de segment en temps
        start_time = start / raw.info['sfreq']
        end_time = end / raw.info['sfreq']
        # Trouver les annotations qui chevauchent ce segment
        segment_annotations = raw.annotations[
            (raw.annotations.onset <= end_time) & 
            ((raw.annotations.onset + raw.annotations.duration) >= start_time)
        ]
        segment_labels = segment_annotations.description.tolist()
        labels.append(segment_labels)
    
    return labels


# Exemple d'utilisation
m = raw.n_times  # nombre total d'échantillons
l = 500  # longueur du segment en échantillons (par exemple 2 secondes si sfreq=250 Hz)
s = 250  # stride en échantillons (par exemple 1 seconde si sfreq=250 Hz)

labels = segment_labels(m, l, s, raw)
print(len(labels))


385


In [5]:
class Segmentation:
    def __init__(self, raw, l, s):
        """
        Initialiser avec les données X, la longueur des segments l, et le décalage s.
        X : numpy array où les lignes sont les variables et les colonnes sont les échantillons.
        """
        self.raw = raw
        self.l = l
        self.s = s
        self.m = raw.get_data().shape[1]

    def segment_labels(self):
        n = (self.m - self.l) // self.s + 1
        labels = []

        for i in range(0,n):
            start = i * self.s
            end = start + self.l
            # Convertir les indices de segment en temps
            start_time = start / self.raw.info['sfreq']
            end_time = end / self.raw.info['sfreq']
            # Trouver les annotations qui chevauchent ce segment
            segment_annotations = self.raw.annotations[
                (self.raw.annotations.onset <= end_time) & 
                ((self.raw.annotations.onset + self.raw.annotations.duration) >= start_time)
            ]
            segment_labels = segment_annotations.description.tolist()
            labels.append(segment_labels)

        return labels


In [6]:
import numpy as np

class CovarianceCalculator:
    def __init__(self, X, l, s):
        """
        Initialiser avec les données X, la longueur des segments l, et le décalage s.
        X : numpy array où les lignes sont les variables et les colonnes sont les échantillons.
        """
        self.X = X
        self.l = l
        self.s = s
        self.C = []
        self.calculate_covariance_matrices()
    
    def calculate_covariance_matrices(self):
        """
        Calculer les matrices de covariance successives à partir des données initiales.
        """
        m = self.X.shape[1]  # nombre de colonnes (échantillons)
        n = (m - self.l) // self.s + 1
        
        for k in range(n):
            start_idx = k * self.s
            end_idx = start_idx + self.l
            segment = self.X[:, start_idx:end_idx]
            cov_matrix = np.cov(segment)
            self.C.append(cov_matrix)
    
    
        
        return labels



In [7]:
from scipy.linalg import sqrtm, logm

class Transformation:
    def __init__(self):
        pass

    def project_to_tangent(self, cov_matrix_current, cov_matrix_next):
        sqrt_B = sqrtm(cov_matrix_next)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, cov_matrix_current), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A

    def transport_to_tangent(self, delta_matrix, tangent_plane_start, tangent_plane_end):
        inv_B = np.linalg.inv(tangent_plane_end)
        A_invB = np.dot(tangent_plane_start, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, delta_matrix), E.T)
        return transported_Delta


In [8]:
class Trajectory:
    def __init__(self, covariance_matrices):
        """
        Initialiser avec une liste de matrices de covariance.
        """
        self.C = covariance_matrices
        self.transformation = Transformation()
        self.Deltas = self.calculate_deltas()

    def calculate_deltas(self):
        """
        Calculer les incréments et les transports successifs pour reconstruire la trajectoire.
        """
        Deltas = {}
        n = len(self.C)
        
        # Calcul des projections directes
        for i in range(n - 1):
            Deltas[(i, i+1)] = self.transformation.project_to_tangent(self.C[i], self.C[i+1])

        # Calcul des transports successifs
        for i in range(n - 1):
            for j in range(i + 2, n):
                Deltas[(i, j)] = self.transformation.transport_to_tangent(Deltas[(i, j-1)], self.C[j-1], self.C[j])

        return Deltas

    def get_delta(self, i, j):
        """
        Obtenir l'incrément ∆i,j
        """
        return self.Deltas.get((i, j), None)


In [9]:
class AntiDevelopment:
    def __init__(self, covariance_matrices):
        """
        Initialiser avec une liste de matrices de covariance.
        """
        self.C = covariance_matrices
        self.transformation = Transformation()
        self.A = self.calculate_antidevelopment()

    def calculate_antidevelopment(self):
        """
        Calculer l'anti-développement de la suite des matrices SPD.
        """
        A = []
        n = len(self.C)
        
        if n > 0:
            # Initialiser A1
            delta_0_1 = self.transformation.project_to_tangent(self.C[0], self.C[1])
            A.append((self.C[0], [delta_0_1]))

            for i in range(1, n - 1):
                # Calcul de ∆i, i+1
                delta_i_i1 = self.transformation.project_to_tangent(self.C[i], self.C[i+1])
                
                # Transport des ∆ précédents
                deltas = []
                for j in range(i):
                    delta = self.transformation.transport_to_tangent(A[i-1][1][j], self.C[i-1], self.C[i])
                    deltas.append(delta)
                deltas.append(delta_i_i1)
                
                A.append((self.C[i], deltas))
        
        return A

    def get_antidevelopment(self, index):
        """
        Obtenir l'anti-développement An pour un index donné.
        """
        if 0 <= index < len(self.A):
            return self.A[index]
        else:
            return None


In [10]:
import numpy as np

class TrajectorySPD:
    def __init__(self, anti_development, stride, frequency):
        """
        Initialiser avec une liste de couples (Cn, deltas), le stride, et la fréquence d'échantillonnage.
        anti_development : liste de tuples (Cn, deltas)
        stride : décalage entre les segments
        frequency : fréquence d'échantillonnage en Hz
        """
        self.A = anti_development
        self.stride = stride
        self.frequency = frequency
        self.time_stamps = self.calculate_time_stamps()
        self.transformation = Transformation()
        self.T = self.calculate_trajectory()

    def calculate_time_stamps(self):
        """
        Calculer les instants d'échantillonnage.
        """
        n = len(self.A)
        t = np.arange(n) * (self.stride / self.frequency)
        return t

    def calculate_trajectory(self):
        """
        Calculer la trajectoire à partir de l'anti-développement et des instants d'échantillonnage.
        """
        T = {}
        k = len(self.A) - 1

        for j in range(k, -1, -1):
            time_diff = self.time_stamps[j] - self.time_stamps[j-1]
            delta_transported = self.transformation.transport_to_tangent(self.A[k][1][j-1], self.A[k][0], np.eye(self.A[k][0].shape[0]))
            T[j] = time_diff * delta_transported 
        
        for i in range(k-1, 0, -1):
            T[i] = T[i+1] + T[i]

        return T

    def get_trajectory(self, index):
        """
        Obtenir la trajectoire à l'index donné.
        """
        return self.T.get(index, None)


In [11]:
dataset = BNCI2014_001()
sessions = dataset.get_data(subjects=[1])
subject = 1
session_name = "1test"
run_name = "0"
raw_test = sessions[subject][session_name][run_name]
raw_test=raw_test.pick_types(eeg=True,exclude='bads')

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [13]:
if __name__ == "__main__":
    
    X = raw.get_data()
    X_test = raw_test.get_data()
    f = raw.info['sfreq'] # fréquence d'échantillonnage
    l = 500  # longueur du segment en échantillons (par exemple 2 secondes si sfreq=250 Hz)
    s = 250  # stride en échantillons (par exemple 1 seconde si sfreq=250 Hz)
    segmentation = Segmentation(raw, l, s)
    labels = segmentation.segment_labels()
    segmentation_test = Segmentation(raw_test, l, s)
    labels_test = segmentation_test.segment_labels()

    # Créer une instance de la classe et calculer les matrices de covariance
    calc = CovarianceCalculator(X, l, s)
    calc_test = CovarianceCalculator(X_test, l, s)

    
    # Créer une instance de Trajectory pour calculer les trajectoires à partir des matrices de covariance
    traj = Trajectory(calc.C)
    traj_test = Trajectory(calc_test.C)
    

    # Créer une instance de AntiDevelopment pour calculer l'anti-développement
    anti_dev = AntiDevelopment(calc.C)
    anti_dev_test = AntiDevelopment(calc_test.C)
    
    # Créer une instance de TrajectorySPD pour calculer la trajectoire
    trajectory_spd = TrajectorySPD(anti_dev.A, s, f)
    trajectory_spd_test = TrajectorySPD(anti_dev_test.A, s, f)
    


In [38]:
antidev =[]
for i in range(0, len(anti_dev.A)):
    antidev.append(anti_dev.A[i][1][-1])

In [39]:
antidev_test = []
for i in range(0, len(anti_dev_test.A)):
    antidev_test.append(anti_dev_test.A[i][1][-1])

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Supposons que vous avez vos données
# x: calc.C est une liste de matrices de covariance (par exemple, une liste de np.ndarray de forme (n_channels, n_channels))
# y: labels est une liste de labels correspondants (par exemple, une liste d'entiers ou de chaînes de caractères)
# calc_test et labels_test sont vos ensembles de test

# Conversion des listes en arrays numpy et vérification des dimensions
X_train = np.array(calc.C)
y_train = np.array(labels_train)
X_test = np.array(calc_test.C)
y_test = np.array(labels_test)

# Vérifiez les dimensions des matrices de covariance
print("X_train shape:", X_train.shape)  # Doit être (n_samples, n_channels, n_channels)
print("X_test shape:", X_test.shape)    # Doit être (n_samples, n_channels, n_channels)
print("y_train shape:", y_train.shape)  # Doit être (n_samples,)
print("y_test shape:", y_test.shape)    # Doit être (n_samples,)


# Création du pipeline avec la transformation en espace tangent et le classifieur SVM
clf = make_pipeline(TangentSpace(metric='riemann'), SVC(kernel='linear'))

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Affichage des résultats
print("Labels prédits : ", y_pred)
print("Labels réels : ", y_test)


X_train shape: (385, 22, 22)
X_test shape: (385, 22, 22)
y_train shape: (385,)
y_test shape: (385,)
Accuracy: 24.94%
Labels prédits :  [3 3 2 1 0 0 0 1 4 0 0 3 3 3 0 0 1 3 2 3 0 0 3 3 0 0 0 1 3 3 0 0 1 3 3 0 0
 0 3 3 3 0 0 1 3 0 3 1 1 3 0 3 0 3 1 1 0 0 1 1 0 1 1 1 3 0 0 2 0 3 3 0 3 3
 3 3 0 1 3 3 3 3 3 3 3 0 3 0 3 3 1 1 0 0 3 3 3 1 3 3 3 3 3 0 1 3 3 1 0 3 0
 1 3 3 3 3 3 0 0 0 3 1 1 3 3 2 3 2 2 3 3 1 0 0 0 2 3 0 3 3 1 1 3 0 0 0 4 0
 0 3 3 3 1 0 1 0 0 3 3 3 0 0 0 0 3 0 0 0 3 2 3 1 3 4 3 3 3 0 0 1 3 4 4 3 0
 0 1 3 3 3 3 3 3 0 0 1 1 0 3 3 0 1 1 3 3 1 2 2 3 3 3 3 3 3 0 0 1 0 3 3 3 3
 0 0 3 3 0 0 2 2 1 1 3 3 3 3 0 2 0 0 0 3 3 3 3 0 0 0 0 3 4 4 4 1 3 3 3 0 3
 0 1 0 3 3 1 1 3 3 0 4 3 3 1 1 1 2 4 3 3 0 0 1 0 0 0 3 3 1 3 0 1 1 3 3 4 0
 3 3 3 1 3 3 0 0 1 2 1 1 1 3 3 3 1 1 1 1 0 0 3 3 3 3 0 0 3 3 3 3 1 1 3 3 1
 1 1 0 0 2 3 3 0 0 2 3 3 4 3 1 3 3 3 2 2 0 3 1 2 1 0 0 0 3 3 3 3 1 3 3 0 2
 0 3 3 3 3 3 3 3 1 3 0 2 2 0 3]
Labels réels :  [3 3 3 3 3 3 0 0 4 4 4 4 4 4 0 4 4 4 4 4 4 0 0 3 3 3 3 3 3 0 0 4 4 

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.classification import MDM
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Supposons que vous avez vos données
# x: calc.C est une liste de matrices de covariance (par exemple, une liste de np.ndarray de forme (n_channels, n_channels))
# y: labels est une liste de labels correspondants (par exemple, une liste d'entiers ou de chaînes de caractères)
# calc_test et labels_test sont vos ensembles de test

# Conversion des listes en arrays numpy et vérification des dimensions
X_train = np.array(calc.C)
y_train = np.array(labels_train)
X_test = np.array(calc_test.C)
y_test = np.array(labels_test)

# Vérifiez les dimensions des matrices de covariance
print("X_train shape:", X_train.shape)  # Doit être (n_samples, n_channels, n_channels)
print("X_test shape:", X_test.shape)    # Doit être (n_samples, n_channels, n_channels)
print("y_train shape:", y_train.shape)  # Doit être (n_samples,)
print("y_test shape:", y_test.shape)    # Doit être (n_samples,)

# Création du pipeline avec le classifieur MDM
clf = make_pipeline( MDM(metric='riemann'))

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Affichage des résultats
print("Labels prédits : ", y_pred)
print("Labels réels : ", y_test)


X_train shape: (385, 22, 22)
X_test shape: (385, 22, 22)
y_train shape: (385,)
y_test shape: (385,)
Accuracy: 27.53%
Labels prédits :  [0 0 0 3 3 3 3 3 0 0 3 3 3 3 3 0 1 2 3 3 0 0 3 3 0 0 3 3 3 3 3 3 3 3 3 3 3
 4 3 3 3 3 3 3 3 3 3 1 1 1 0 3 3 3 3 0 0 1 1 2 2 1 1 1 0 3 3 4 4 3 0 0 0 0
 2 1 1 1 0 0 3 3 3 3 0 0 0 1 3 3 1 2 2 0 0 3 3 4 3 3 4 4 3 3 1 3 3 2 3 3 2
 2 2 1 1 3 4 4 3 0 0 2 2 2 2 2 2 2 2 3 3 2 2 2 2 2 0 0 1 1 1 1 0 0 0 0 4 4
 1 1 1 1 1 3 3 3 3 3 3 3 3 3 1 1 3 3 0 0 3 3 3 3 3 4 0 0 0 0 3 3 3 3 3 4 0
 0 3 3 3 3 3 3 3 1 1 2 2 2 3 3 3 4 2 2 3 3 2 3 3 3 3 3 3 3 0 0 0 0 3 2 3 2
 0 0 0 3 3 3 2 2 0 0 0 0 3 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 3 3 3 1 3
 3 3 3 3 3 2 1 0 0 3 4 4 0 0 1 1 2 4 3 3 3 1 1 1 0 4 3 3 3 3 0 2 2 2 3 4 3
 3 3 3 1 1 3 1 3 2 2 1 1 0 3 3 3 0 0 0 3 3 3 3 3 3 0 0 0 3 2 2 3 3 3 3 3 3
 2 2 3 2 2 3 3 2 2 2 2 3 3 0 0 3 0 0 0 0 0 3 3 3 1 1 2 2 2 3 3 2 2 3 3 0 0
 1 3 2 2 2 2 3 2 2 3 3 2 2 3 0]
Labels réels :  [3 3 3 3 3 3 0 0 4 4 4 4 4 4 0 4 4 4 4 4 4 0 0 3 3 3 3 3 3 0 0 4 4 

In [43]:
# Importation des bibliothèques nécessaires
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from pyriemann.classification import MDM
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Supposons que vous avez vos données
# x: calc.C est une liste de matrices de covariance (par exemple, une liste de np.ndarray de forme (n_channels, n_channels))
# y: labels est une liste de labels correspondants (par exemple, une liste d'entiers ou de chaînes de caractères)
# calc_test et labels_test sont vos ensembles de test

# Conversion des listes en arrays numpy et vérification des dimensions
X_train = np.array(antidev)
y_train = np.array(labels_train)
X_test = np.array(antidev_test)
y_test = np.array(labels_test)

# Vérifiez les dimensions des matrices de covariance
print("X_train shape:", X_train.shape)  # Doit être (n_samples, n_channels, n_channels)
print("X_test shape:", X_test.shape)    # Doit être (n_samples, n_channels, n_channels)
print("y_train shape:", y_train.shape)  # Doit être (n_samples,)
print("y_test shape:", y_test.shape)    # Doit être (n_samples,)


# Création du pipeline avec la transformation en espace tangent et le classifieur SVM
clf = make_pipeline(MDM(metric='riemann'))

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Affichage des résultats
print("Labels prédits : ", y_pred)
print("Labels réels : ", y_test)


X_train shape: (384, 22, 22)
X_test shape: (384, 22, 22)
y_train shape: (384,)
y_test shape: (384,)


c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\pyriemann\utils\base.py:18: RuntimeWarning: invalid value encountered in sqrt
  eigvals = operator(eigvals)
c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\pyriemann\utils\base.py:73: RuntimeWarning: invalid value encountered in sqrt
  def isqrt(x): return 1. / np.sqrt(x)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from pyriemann.classification import MDM
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Supposons que vous avez vos données
# x: calc.C est une liste de matrices de covariance (par exemple, une liste de np.ndarray de forme (n_channels, n_channels))
# y: labels est une liste de labels correspondants (par exemple, une liste d'entiers ou de chaînes de caractères)
# calc_test et labels_test sont vos ensembles de test

# Conversion des listes en arrays numpy et vérification des dimensions
X_train = np.array(trajectoire)
y_train = np.array(labels_train)
X_test = np.array(trajectoire_test)
y_test = np.array(labels_test)

# Vérifiez les dimensions des matrices de covariance
print("X_train shape:", X_train.shape)  # Doit être (n_samples, n_channels, n_channels)
print("X_test shape:", X_test.shape)    # Doit être (n_samples, n_channels, n_channels)
print("y_train shape:", y_train.shape)  # Doit être (n_samples,)
print("y_test shape:", y_test.shape)    # Doit être (n_samples,)


# Création du pipeline avec la transformation en espace tangent et le classifieur SVM
clf = make_pipeline(MDM(metric='riemann'))

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Affichage des résultats
print("Labels prédits : ", y_pred)
print("Labels réels : ", y_test)


X_train shape: (384, 22, 22)
X_test shape: (384, 22, 22)
y_train shape: (384,)
y_test shape: (384,)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Supposons que vous avez vos données
# x: calc.C est une liste de matrices de covariance (par exemple, une liste de np.ndarray de forme (n_channels, n_channels))
# y: labels est une liste de labels correspondants (par exemple, une liste d'entiers ou de chaînes de caractères)
# calc_test et labels_test sont vos ensembles de test

# Conversion des listes en arrays numpy et vérification des dimensions
X_train = np.array(trajectoire)
y_train = np.array(labels_train)
X_test = np.array(trajectoire_test)
y_test = np.array(labels_test)

# Vérifiez les dimensions des matrices de covariance
print("X_train shape:", X_train.shape)  # Doit être (n_samples, n_channels, n_channels)
print("X_test shape:", X_test.shape)    # Doit être (n_samples, n_channels, n_channels)
print("y_train shape:", y_train.shape)  # Doit être (n_samples,)
print("y_test shape:", y_test.shape)    # Doit être (n_samples,)


# Création du pipeline avec la transformation en espace tangent et le classifieur SVM
clf = make_pipeline(TangentSpace(metric='riemann'), SVC(kernel='linear'))

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Affichage des résultats
print("Labels prédits : ", y_pred)
print("Labels réels : ", y_test)


X_train shape: (384, 22, 22)
X_test shape: (384, 22, 22)
y_train shape: (384,)
y_test shape: (384,)


c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\pyriemann\utils\base.py:18: RuntimeWarning: invalid value encountered in sqrt
  eigvals = operator(eigvals)
c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\pyriemann\utils\base.py:73: RuntimeWarning: invalid value encountered in sqrt
  def isqrt(x): return 1. / np.sqrt(x)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [ ]:
trajectoire = []
for i in range(0, len(trajectory_spd.T)):
    trajectoire.append(trajectory_spd.T[i]) 


In [ ]:
trajectoire_test = []
for i in range(0, len(trajectory_spd_test.T)):
    trajectoire_test.append(trajectory_spd_test.T[i])

In [42]:
# Dictionnaire de mappage des labels
label_mapping = {
    '': 0,
    'tongue': 1,
    'feet': 2,
    'left_hand': 3,
    'right_hand': 4
}

# Conversion des labels en nombres
labels_train = [label_mapping[label[0]] if label else 0 for label in labels]
labels_test = [label_mapping[label[0]] if label else 0 for label in labels_test]    

# Conversion en array numpy
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

print(labels_train)


[1 1 1 1 1 1 0 0 2 2 2 2 2 2 0 4 4 4 4 4 4 0 0 3 3 3 3 3 3 0 0 3 3 3 3 3 3
 0 0 4 4 4 4 4 4 0 0 2 2 2 2 2 2 0 0 1 1 1 1 1 1 0 0 4 4 4 4 4 4 0 0 2 2 2
 2 2 2 0 0 3 3 3 3 3 3 0 3 3 3 3 3 3 0 0 3 3 3 3 3 3 0 0 1 1 1 1 1 1 0 0 4
 4 4 4 4 4 0 0 4 4 4 4 4 4 0 0 3 3 3 3 3 3 0 0 0 3 3 3 3 3 3 0 0 2 2 2 2 2
 2 0 0 3 3 3 3 3 3 0 0 4 4 4 4 4 4 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 2 2
 2 2 2 2 0 0 3 3 3 3 3 3 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 4 4 4 4 4 4 0
 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 4 4 4 4 4 4 0 0 3 3 3 3 3 3 0 0 0 4 4
 4 4 4 4 0 0 2 2 2 2 2 2 0 0 2 2 2 2 2 2 0 0 2 2 2 2 2 2 0 0 1 1 1 1 1 1 0
 0 2 2 2 2 2 2 0 0 3 3 3 3 3 3 0 0 1 1 1 1 1 1 0 0 4 4 4 4 4 4 0 0 2 2 2 2
 2 2 0 0 4 4 4 4 4 4 0 0 2 2 2 2 2 2 2 0 1 1 1 1 1 1 0 0 4 4 4 4 4 4 0 0 2
 2 2 2 2 2 0 0 0 3 3 3 3 3 3]


In [41]:
labels.pop(-1)
labels_test.pop(-1)

[]

In [ ]:
print(len(labels))
print(len(calc.C))
print(len(anti_dev.A))
print(len(trajectory_spd.T))

385
385
384
384


In [ ]:
if __name__ == "__main__":
    
    X = raw.get_data()
    f = 250.0  # fréquence d'échantillonnage
    l = 500  # longueur du segment en échantillons (par exemple 2 secondes si sfreq=250 Hz)
    s = 250  # stride en échantillons (par exemple 1 seconde si sfreq=250 Hz)
    segmentation = Segmentation(raw, l, s)
    labels = segmentation.segment_labels()

    # Créer une instance de la classe et calculer les matrices de covariance
    calc = CovarianceCalculator(X, l, s)

    
    # Créer une instance de Trajectory pour calculer les trajectoires à partir des matrices de covariance
    traj = Trajectory(calc.C)
    
    # Afficher les deltas calculés
    #for key, delta in traj.Deltas.items():
    #    print(f"Delta {key}:")
    #    print(delta)

    # Créer une instance de AntiDevelopment pour calculer l'anti-développement
    anti_dev = AntiDevelopment(calc.C)
    # Afficher l'anti-développement calculé
    #for index, anti_dev_item in enumerate(anti_dev.A):
    #    print(f"Anti-développement A{index}:")
    #    print("Matrice de covariance:", anti_dev_item[0])
    #    print("Deltas:", anti_dev_item[1])
    
    # Créer une instance de TrajectorySPD pour calculer la trajectoire
    trajectory_spd = TrajectorySPD(anti_dev.A, s, f)
    
    # Afficher la trajectoire calculée
    for index, traj in trajectory_spd.T.items():
        print(f"Trajectoire T{index}:")
        print(traj)
    
